In [ ]:
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
from shapely.wkt import loads
import geopandas as gpd

In [3]:
# Conectar ao banco de dados PostgreSQL
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="postgres",
    host="localhost",
    port="5432"
)

### Plot Grid and Points for 606

In [7]:
specific_line = '606'  # Replace with the desired line identifier

# Query to retrieve grid data for the specific line
query_grid = f"""
SELECT id, ST_AsText(geom) AS geom
FROM grid_cells_linha
WHERE linha = '{specific_line}'
"""

# Query to retrieve point data for the specific line
query_points = f"""
SELECT ST_AsText(geom) AS geom
FROM public.vehicle_tracking
WHERE linha = '{specific_line}' AND datahora >= '20240508_0000' AND datahora < '20240509_0000'
"""

# Read data into pandas DataFrames
df_grid = pd.read_sql(query_grid, conn)
df_points = pd.read_sql(query_points, conn)

# Close the database connection
conn.close()


C:\Users\mwuillau\AppData\Local\Temp\ipykernel_25276\3679883692.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_grid = pd.read_sql(query_grid, conn)


In [4]:
# Convert the geometry data to GeoDataFrames
df_grid['geometry'] = df_grid['geom'].apply(loads)
gdf_grid = gpd.GeoDataFrame(df_grid, geometry='geometry', crs="EPSG:4326")

df_points['geometry'] = df_points['geom'].apply(loads)
gdf_points = gpd.GeoDataFrame(df_points, geometry='geometry', crs="EPSG:4326")

# Plot the grid and the points
fig, ax = plt.subplots(figsize=(10, 10))

# Plot grid cells
gdf_grid.plot(ax=ax, color='none', edgecolor='blue', linewidth=0.5)

# Plot points
gdf_points.plot(ax=ax, color='red', markersize=1, alpha=0.7)

# Customize plot
plt.title(f'Grid and Points for Line {specific_line}')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.grid(True)

# Show plot
plt.show()

NameError: name 'df_grid' is not defined

In [ ]:



# Consultar os dados de contagem de pontos de 00h às 5h
query_00_05 = """
SELECT linha, point_count
FROM grid_point_counts_00_05
"""

# Ler os dados em um DataFrame do pandas
df_00_05 = pd.read_sql(query_00_05, conn)

# Consultar os dados de contagem de pontos de 5h às 24h
query_05_24 = """
SELECT linha, point_count
FROM grid_point_counts_05_24
"""

# Ler os dados em um DataFrame do pandas
df_05_24 = pd.read_sql(query_05_24, conn)

# Fechar a conexão com o banco de dados
conn.close()

# Plotar o histograma para cada faixa de tempo
def plot_histogram(df, title):
    plt.figure(figsize=(10, 6))
    plt.hist(df['point_count'], bins=30, edgecolor='black', alpha=0.7)
    plt.title(title)
    plt.xlabel('Número de Pontos')
    plt.ylabel('Frequência')
    plt.yscale('log')  # Usar escala logarítmica